# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import pickle
import helper
import problem_unittests as tests

with open('most_common_indexes.p', 'rb') as f:
    indexes = pickle.load(f)

source_path = 'data/es-en/europarl-v7.es-en.en'
target_path = 'data/es-en/europarl-v7.es-en.es'
source_text, target_text = helper.load_data(source_path, target_path, indexes, helper.preprocess)

#source_text, target_text = helper.preprocess(source_text, target_text)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({w: None for l in source_text for w in l})))

sentences = source_text
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print(''.join(source_text[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('Spanish sentences {} to {}:'.format(*view_sentence_range))
print(''.join(target_text[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 95
Number of sentences: 37816
Average number of words in a sentence: 9.208139411889148

English sentences 0 to 10:
according to its president it is in a position to do sowe therefore respect whatever parliament may decidewe shall now hear mr wurtz speaking against this requestit is not a lot to asksafety advisers for the transport of dangerous goodsi would urge you to endorse thisi congratulate him on his excellent reportthis is a pity in a sensei would like to mention one final point

Spanish sentences 0 to 10:
según palabras de su presidente está en condiciones de hacerlopor lo tanto nosotros respetamos lo que en este sentido pueda decidir el parlamentoescuchemos ahora al sr wurtz que hablará en contra de la peticióny él hace lo mismono es pedir demasiadoconsejeros de seguridad para el transporte de mercancías peligrosaspido la conformidad de ustedesle felicito por su excelente informees una pena en cierto sentidopermítanme que mencio

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    source_id_text = [
        [source_vocab_to_int[w] for w in line.split()] for line in source_text]
    
    id_eos = target_vocab_to_int['<EOS>']
    target_id_text = [
        [target_vocab_to_int[w] for w in line.split()] + [id_eos] for line in target_text]
    
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
#tests.test_text_to_ids(text_to_ids)

"\nDON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE\n"

### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
source_text, target_text, source_vocab_to_int, target_vocab_to_int, source_int_to_vocab, target_int_to_vocab = \
helper.process(source_text, target_text, text_to_ids)

    
helper.save_data(
    source_text, target_text,
    source_vocab_to_int, target_vocab_to_int,
    source_int_to_vocab, target_int_to_vocab
)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()

In [6]:
' '.join([source_int_to_vocab[i] for i in source_int_text[1:2][0]])

'we therefore respect whatever parliament may decide'

In [7]:
' '.join([target_int_to_vocab[i] for i in target_int_text[1:2][0]])

'por lo tanto nosotros respetamos lo que en este sentido pueda decidir el parlamento <EOS>'

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [9]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='target')
    l_rate = tf.placeholder(tf.float32)
    
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    seq_length = tf.placeholder(tf.int32, [None], name='target_sequence_length')
    max_targ_len = tf.reduce_max(seq_length, name='max_target_len')
    source = tf.placeholder(tf.int32, [None], name='source_sequence_length')
    
    return inputs, targets, l_rate, keep_prob, seq_length, max_targ_len, source


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [10]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    x = tf.strided_slice(target_data, [0,0], [batch_size, -1], [1,1])
    y = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), x], 1)
    return y

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [11]:
from imp import reload
reload(tests)

def make_cell(rnn_size, keep_prob):
    lstm = tf.contrib.rnn.LSTMCell(
        rnn_size,
        initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2)
    )
    return tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """ 
    enc_embed_input = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    
    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size, keep_prob) for _ in range(num_layers)])
    
    enc_output, enc_state = tf.nn.dynamic_rnn(
        enc_cell, 
        enc_embed_input, 
        sequence_length=source_sequence_length, 
        dtype=tf.float32
    )
    return enc_output, enc_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [12]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    helper = tf.contrib.seq2seq.TrainingHelper(
        inputs=dec_embed_input,
        sequence_length=target_sequence_length,
    )
    
    decoder = tf.contrib.seq2seq.BasicDecoder(
        dec_cell,
        helper,
        encoder_state,
        output_layer
    ) 
    
    output, _ = tf.contrib.seq2seq.dynamic_decode(
        decoder,
        impute_finished=True,
        maximum_iterations=max_summary_length
    )
    return output

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [13]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    start_tokens = tf.tile(
        tf.constant(
            [start_of_sequence_id], 
            dtype=tf.int32
        ),
        [batch_size], 
        name='start_tokens'
    )
    
    helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
        dec_embeddings,
        start_tokens,
        end_of_sequence_id
    )
    
    decoder = tf.contrib.seq2seq.BasicDecoder(
        dec_cell,
        helper,
        encoder_state,
        output_layer
    )
    
    decoder_output, _ = tf.contrib.seq2seq.dynamic_decode(
        decoder,
        impute_finished=True,
        maximum_iterations=max_target_sequence_length
    )
    return decoder_output

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [14]:
from tensorflow.python.layers.core import Dense

def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size, keep_prob) for _ in range(num_layers)])

    output_layer = Dense(
        target_vocab_size,
        kernel_initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1)
    )
    
    with tf.variable_scope("decode"):   
        train_dec = decoding_layer_train(
            encoder_state,
            dec_cell,
            dec_embed_input,
            target_sequence_length,
            max_target_sequence_length,
            output_layer,
            keep_prob
        )

    with tf.variable_scope("decode", reuse=True):    
        infer_dec = decoding_layer_infer(
            encoder_state,
            dec_cell,
            dec_embeddings,
            target_vocab_to_int['<GO>'],
            target_vocab_to_int['<EOS>'],
            max_target_sequence_length,
            len(target_vocab_to_int),
            output_layer,
            batch_size,
            keep_prob
        ) 
    return train_dec, infer_dec

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Apply embedding to the input data for the encoder.
- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Apply embedding to the target data for the decoder.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [15]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    _, enc_state = encoding_layer(
        input_data, 
        rnn_size, 
        num_layers,
        keep_prob,
        source_sequence_length,
        source_vocab_size, 
        enc_embedding_size
    )
    
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    training_dec, inference_dec = decoding_layer(
        dec_input,
        enc_state,
        target_sequence_length,
        max_target_sentence_length,
        rnn_size,
        num_layers,
        target_vocab_to_int,
        target_vocab_size,
        batch_size,
        keep_prob,
        dec_embedding_size
    ) 
    
    return training_dec, inference_dec


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [24]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 64
# RNN Size
rnn_size = 256
# Number of Layers
num_layers = 3
# Embedding Size
encoding_embedding_size = 256
decoding_embedding_size = 256
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.6
display_step = 10

### Build the Graph
Build the graph using the neural network you implemented.

In [25]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [26]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [27]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]

(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(
    get_batches(
        valid_source,
        valid_target,
        batch_size,
        source_vocab_to_int['<PAD>'],
        target_vocab_to_int['<PAD>']
    )
)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:

                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})

                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: \
                {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'.format(
                    epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss
                ))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')


Epoch   0 Batch   10/590 - Train Accuracy:                 0.5340, Validation Accuracy: 0.4482, Loss: 4.6785
Epoch   0 Batch   20/590 - Train Accuracy:                 0.5533, Validation Accuracy: 0.4457, Loss: 3.8235
Epoch   0 Batch   30/590 - Train Accuracy:                 0.6124, Validation Accuracy: 0.4482, Loss: 3.1473
Epoch   0 Batch   40/590 - Train Accuracy:                 0.4911, Validation Accuracy: 0.4465, Loss: 3.8379
Epoch   0 Batch   50/590 - Train Accuracy:                 0.5949, Validation Accuracy: 0.4572, Loss: 3.0015
Epoch   0 Batch   60/590 - Train Accuracy:                 0.6695, Validation Accuracy: 0.4581, Loss: 2.3810
Epoch   0 Batch   70/590 - Train Accuracy:                 0.5699, Validation Accuracy: 0.4622, Loss: 3.0644
Epoch   0 Batch   80/590 - Train Accuracy:                 0.5482, Validation Accuracy: 0.4597, Loss: 3.0864
Epoch   0 Batch   90/590 - Train Accuracy:                 0.5841, Validation Accuracy: 0.4613, Loss: 2.8735
Epoch   0 Batch  10

Epoch   1 Batch  190/590 - Train Accuracy:                 0.5015, Validation Accuracy: 0.4202, Loss: 2.5861
Epoch   1 Batch  200/590 - Train Accuracy:                 0.4745, Validation Accuracy: 0.4178, Loss: 2.8278
Epoch   1 Batch  210/590 - Train Accuracy:                 0.5394, Validation Accuracy: 0.4186, Loss: 2.5642
Epoch   1 Batch  220/590 - Train Accuracy:                 0.5403, Validation Accuracy: 0.4104, Loss: 2.5145
Epoch   1 Batch  230/590 - Train Accuracy:                 0.4418, Validation Accuracy: 0.4243, Loss: 3.1380
Epoch   1 Batch  240/590 - Train Accuracy:                 0.8513, Validation Accuracy: 0.4219, Loss: 0.8375
Epoch   1 Batch  250/590 - Train Accuracy:                 0.5531, Validation Accuracy: 0.4211, Loss: 2.4420
Epoch   1 Batch  260/590 - Train Accuracy:                 0.5709, Validation Accuracy: 0.4112, Loss: 2.3844
Epoch   1 Batch  270/590 - Train Accuracy:                 0.6578, Validation Accuracy: 0.4276, Loss: 1.8324
Epoch   1 Batch  28

Epoch   2 Batch  370/590 - Train Accuracy:                 0.4115, Validation Accuracy: 0.4260, Loss: 2.8296
Epoch   2 Batch  380/590 - Train Accuracy:                 0.6705, Validation Accuracy: 0.4120, Loss: 1.7214
Epoch   2 Batch  390/590 - Train Accuracy:                 0.4137, Validation Accuracy: 0.4383, Loss: 3.0111
Epoch   2 Batch  400/590 - Train Accuracy:                 0.4918, Validation Accuracy: 0.4128, Loss: 2.4015
Epoch   2 Batch  410/590 - Train Accuracy:                 0.5673, Validation Accuracy: 0.4268, Loss: 2.2051
Epoch   2 Batch  420/590 - Train Accuracy:                 0.3327, Validation Accuracy: 0.4597, Loss: 3.1883
Epoch   2 Batch  430/590 - Train Accuracy:                 0.5845, Validation Accuracy: 0.4383, Loss: 2.0984
Epoch   2 Batch  440/590 - Train Accuracy:                 0.5462, Validation Accuracy: 0.4367, Loss: 2.3612
Epoch   2 Batch  450/590 - Train Accuracy:                 0.4879, Validation Accuracy: 0.4301, Loss: 2.6870
Epoch   2 Batch  46

Epoch   3 Batch  550/590 - Train Accuracy:                 0.4595, Validation Accuracy: 0.4211, Loss: 2.5023
Epoch   3 Batch  560/590 - Train Accuracy:                 0.6795, Validation Accuracy: 0.4572, Loss: 1.4581
Epoch   3 Batch  570/590 - Train Accuracy:                 0.4591, Validation Accuracy: 0.4507, Loss: 2.4061
Epoch   3 Batch  580/590 - Train Accuracy:                 0.7653, Validation Accuracy: 0.4449, Loss: 1.0426
Epoch   4 Batch   10/590 - Train Accuracy:                 0.5523, Validation Accuracy: 0.4507, Loss: 2.0608
Epoch   4 Batch   20/590 - Train Accuracy:                 0.5582, Validation Accuracy: 0.4762, Loss: 2.1279
Epoch   4 Batch   30/590 - Train Accuracy:                 0.6034, Validation Accuracy: 0.4688, Loss: 1.8325
Epoch   4 Batch   40/590 - Train Accuracy:                 0.5052, Validation Accuracy: 0.4342, Loss: 2.2596
Epoch   4 Batch   50/590 - Train Accuracy:                 0.6065, Validation Accuracy: 0.4638, Loss: 1.8995
Epoch   4 Batch   6

Epoch   5 Batch  150/590 - Train Accuracy:                 0.4188, Validation Accuracy: 0.4112, Loss: 2.4842
Epoch   5 Batch  160/590 - Train Accuracy:                 0.5097, Validation Accuracy: 0.4169, Loss: 1.9845
Epoch   5 Batch  170/590 - Train Accuracy:                 0.4866, Validation Accuracy: 0.4482, Loss: 2.2009
Epoch   5 Batch  180/590 - Train Accuracy:                 0.5320, Validation Accuracy: 0.4613, Loss: 1.9840
Epoch   5 Batch  190/590 - Train Accuracy:                 0.5365, Validation Accuracy: 0.4457, Loss: 1.8384
Epoch   5 Batch  200/590 - Train Accuracy:                 0.5082, Validation Accuracy: 0.4688, Loss: 2.0546
Epoch   5 Batch  210/590 - Train Accuracy:                 0.5794, Validation Accuracy: 0.4712, Loss: 1.9252
Epoch   5 Batch  220/590 - Train Accuracy:                 0.5765, Validation Accuracy: 0.4482, Loss: 1.8443
Epoch   5 Batch  230/590 - Train Accuracy:                 0.4510, Validation Accuracy: 0.4482, Loss: 2.3118
Epoch   5 Batch  24

Epoch   6 Batch  330/590 - Train Accuracy:                 0.4828, Validation Accuracy: 0.4638, Loss: 2.3095
Epoch   6 Batch  340/590 - Train Accuracy:                 0.5227, Validation Accuracy: 0.4745, Loss: 2.0060
Epoch   6 Batch  350/590 - Train Accuracy:                 0.4955, Validation Accuracy: 0.4556, Loss: 1.9911
Epoch   6 Batch  360/590 - Train Accuracy:                 0.6568, Validation Accuracy: 0.4178, Loss: 1.4573
Epoch   6 Batch  370/590 - Train Accuracy:                 0.4457, Validation Accuracy: 0.4515, Loss: 2.2271
Epoch   6 Batch  380/590 - Train Accuracy:                 0.6875, Validation Accuracy: 0.4655, Loss: 1.4222
Epoch   6 Batch  390/590 - Train Accuracy:                 0.4128, Validation Accuracy: 0.4548, Loss: 2.3197
Epoch   6 Batch  400/590 - Train Accuracy:                 0.5171, Validation Accuracy: 0.4548, Loss: 1.8873
Epoch   6 Batch  410/590 - Train Accuracy:                 0.5685, Validation Accuracy: 0.4416, Loss: 1.7382
Epoch   6 Batch  42

Epoch   7 Batch  510/590 - Train Accuracy:                 0.7997, Validation Accuracy: 0.4416, Loss: 0.8341
Epoch   7 Batch  520/590 - Train Accuracy:                 0.5335, Validation Accuracy: 0.4663, Loss: 1.9503
Epoch   7 Batch  530/590 - Train Accuracy:                 0.4414, Validation Accuracy: 0.4424, Loss: 2.1452
Epoch   7 Batch  540/590 - Train Accuracy:                 0.5133, Validation Accuracy: 0.4556, Loss: 1.8664
Epoch   7 Batch  550/590 - Train Accuracy:                 0.5064, Validation Accuracy: 0.4539, Loss: 2.0693
Epoch   7 Batch  560/590 - Train Accuracy:                 0.6833, Validation Accuracy: 0.4482, Loss: 1.2581
Epoch   7 Batch  570/590 - Train Accuracy:                 0.4814, Validation Accuracy: 0.4613, Loss: 1.9740
Epoch   7 Batch  580/590 - Train Accuracy:                 0.7854, Validation Accuracy: 0.4704, Loss: 0.9189
Epoch   8 Batch   10/590 - Train Accuracy:                 0.5577, Validation Accuracy: 0.4515, Loss: 1.7044
Epoch   8 Batch   2

Epoch   9 Batch  110/590 - Train Accuracy:                 0.6034, Validation Accuracy: 0.4441, Loss: 1.5693
Epoch   9 Batch  120/590 - Train Accuracy:                 0.4963, Validation Accuracy: 0.4704, Loss: 1.9960
Epoch   9 Batch  130/590 - Train Accuracy:                 0.6223, Validation Accuracy: 0.4564, Loss: 1.3754
Epoch   9 Batch  140/590 - Train Accuracy:                 0.4671, Validation Accuracy: 0.4474, Loss: 2.0176
Epoch   9 Batch  150/590 - Train Accuracy:                 0.4461, Validation Accuracy: 0.4482, Loss: 2.0153
Epoch   9 Batch  160/590 - Train Accuracy:                 0.5551, Validation Accuracy: 0.4605, Loss: 1.6280
Epoch   9 Batch  170/590 - Train Accuracy:                 0.4792, Validation Accuracy: 0.4391, Loss: 1.8195
Epoch   9 Batch  180/590 - Train Accuracy:                 0.5238, Validation Accuracy: 0.4663, Loss: 1.6392
Epoch   9 Batch  190/590 - Train Accuracy:                 0.5499, Validation Accuracy: 0.4762, Loss: 1.5195
Epoch   9 Batch  20

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [28]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [29]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [30]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    sentence = sentence.lower()
    unk_id = vocab_to_int.get('<UNK>')
    ids = [vocab_to_int.get(word, unk_id) for word in sentence.split()]
    return ids


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [31]:
translate_sentence = 'resumption of the session .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [5448, 5908, 1261, 14415, 2]
  English Words: ['resumption', 'of', 'the', 'session', '<UNK>']

Prediction
  Word Ids:      [14594, 7991, 10191, 5651, 7991, 11232, 1]
  French Words: reanudación del artículo 137 del reglamento <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.